In [1]:
import pandas as pd
import numpy as np
import time
import itertools

In [2]:
dogs = pd.read_csv("dogs.csv")

In [3]:
dogs.head()

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
0,46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,Dog,Dog,American Staffordshire Terrier,Mixed Breed,True,False,White / Cream,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,Harley is not sure how he wound up at shelter ...,70,124.81
1,46042002,NV163,https://www.petfinder.com/dog/biggie-46042002/...,Dog,Dog,Pit Bull Terrier,Mixed Breed,True,False,Brown / Chocolate,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,6 year old Biggie has lost his home and really...,49,122.07
2,46040898,NV99,https://www.petfinder.com/dog/ziggy-46040898/n...,Dog,Dog,Shepherd,NaN,False,False,Brindle,...,Mesquite,NV,89027,US,89009,2019-09-20,Dog,Approx 2 years old.\n Did I catch your eye? I ...,87,281.51
3,46039877,NV202,https://www.petfinder.com/dog/gypsy-46039877/n...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,...,Pahrump,NV,89048,US,89009,2019-09-20,Dog,NaN,62,145.83
4,46039306,NV184,https://www.petfinder.com/dog/theo-46039306/nv...,Dog,Dog,Dachshund,NaN,False,False,NaN,...,Henderson,NV,89052,US,89009,2019-09-20,Dog,Theo is a friendly dachshund mix who gets alon...,93,241.09


In [4]:
travels = pd.read_csv("dogTravel.csv")

In [5]:
travels.head()

,index,id,contact_city,contact_state,description,found,manual,remove,still_there
0,0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
1,1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Abacos,Bahamas,NaN,NaN
2,2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Adam,Maryland,NaN,NaN
3,3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN


Prepping the dataset

In [6]:
#c'è un errore nel CSV, le righe che non hanno come status 'adoptable' hanno i valori delle colonne da status a accessed sfalsati di 1, e in corrispondenza di 
#accessed c'è un Nan. trasliamo tutto di 1 e mettiamo un NaN in corrispondenza di status
wrongly_formatted = dogs.loc[dogs['status'] != 'adoptable'].loc[:, 'status':'accessed'].shift(axis=1)

#serve aggiungere 1 altrimenti considera fino al valore di accessed_index escluso
accessed_index = dogs.columns.get_loc('accessed') + 1
status_index = dogs.columns.get_loc('status')

counter = 0

for index in wrongly_formatted.index.tolist():
    dogs.iloc[index, status_index:accessed_index] = wrongly_formatted.iloc[counter, :]
    counter += 1

In [7]:
#correzione di un invertimento tra le colonne contact_state e contact_city di travels
travels.loc[(travels['contact_state'] == '17325'),'contact_state']='PA'
travels.loc[travels['contact_city']=='PA', 'contact_city'] = '17325'

In [8]:
#NB è una forma desueta per riferirsi allo stato del Nebraska, che dunque sostituiamo con NE
travels.loc[(travels['contact_state'] == 'NB'),'contact_state']='NE'
dogs.loc[(dogs['contact_state'] == 'NB'),'contact_state']='NE'

In [9]:
#servirà al punto 12 per comparare le descrizioni
dogs['description'] = dogs['description'].fillna('') 

1. Extract all dogs with status that is not adoptable

In [10]:
dogs[dogs['status'] != 'adoptable'][['id','status']]

,id,status
644,41330726,None
5549,38169117,None
10888,45833989,None
11983,45515547,None
12495,45294115,None
12600,45229004,None
12613,45227052,None
17619,45569380,None
18611,44694387,None
19747,36978896,None


2. For each (primary) breed, determine the number of dogs

In [11]:
primary_breed = dogs.groupby('breed_primary').count()[['id']]
primary_breed = primary_breed.rename(columns = {'id':'count'})

In [12]:
primary_breed

,count
breed_primary,
Affenpinscher,17
Afghan Hound,4
Airedale Terrier,19
Akbash,3
Akita,181
...,...
Wirehaired Pointing Griffon,1
Wirehaired Terrier,60
Xoloitzcuintli / Mexican Hairless,11


3. For each (primary) breed, determine the ratio between the number of dogs of Mixed Breed and those not of Mixed Breed. Hint: look at the secondary_breed.

In [13]:
breeds = pd.DataFrame()

In [14]:
#un cane è purarazza se ha nan come breed_secondary
#al contrario, è mixed breed se la sua breed_secondary non è nulla
breeds['mixed_breed']=(dogs[dogs['breed_secondary'].notna()].groupby('breed_primary').size())
breeds['purebred']=(dogs[dogs['breed_secondary'].isna()].groupby('breed_primary').size())

In [15]:
#riempire i valori na con 0
breeds['mixed_breed'] = breeds['mixed_breed'].fillna(0)
breeds['purebred'] = breeds['purebred'].fillna(0)

In [16]:
#per questioni di pulizia, converto ad intero la colonna purebred
breeds['purebred'] = breeds['purebred'].astype(int)

In [17]:
breeds['ratio']=(breeds['mixed_breed']/breeds['purebred'])

In [18]:
breeds

,mixed_breed,purebred,ratio
breed_primary,,,
Affenpinscher,2,15,0.133333
Afghan Hound,1,3,0.333333
Airedale Terrier,9,10,0.900000
Akita,52,129,0.403101
Alaskan Malamute,14,58,0.241379
...,...,...,...
Wirehaired Dachshund,2,2,1.000000
Wirehaired Terrier,18,42,0.428571
Xoloitzcuintli / Mexican Hairless,4,7,0.571429


4. For each (primary) breed, determine the earliest and the latest posted timestamp.


In [19]:
#formattare la data di posted 
dogs['posted'] = pd.to_datetime(dogs['posted'], format='%Y-%m-%d')

In [20]:
dogs[['breed_primary', 'posted']].groupby('breed_primary')['posted'].min()

breed_primary
Affenpinscher                       2012-03-08 10:27:33+00:00
Afghan Hound                        2017-06-29 23:28:51+00:00
Airedale Terrier                    2014-06-13 12:59:36+00:00
Akbash                              2019-07-21 00:35:59+00:00
Akita                               2012-03-03 09:31:08+00:00
                                               ...           
Wirehaired Pointing Griffon         2016-06-29 20:03:55+00:00
Wirehaired Terrier                  2012-11-27 14:07:54+00:00
Xoloitzcuintli / Mexican Hairless   2007-02-01 00:00:00+00:00
Yellow Labrador Retriever           2010-05-31 00:00:00+00:00
Yorkshire Terrier                   2012-02-04 17:14:52+00:00
Name: posted, Length: 216, dtype: datetime64[ns, UTC]

In [21]:
dogs[['breed_primary', 'posted']].groupby('breed_primary')['posted'].max()

breed_primary
Affenpinscher                       2019-09-14 10:10:51+00:00
Afghan Hound                        2019-07-27 00:38:48+00:00
Airedale Terrier                    2019-09-19 18:40:39+00:00
Akbash                              2019-08-23 17:11:04+00:00
Akita                               2019-09-20 15:19:57+00:00
                                               ...           
Wirehaired Pointing Griffon         2016-06-29 20:03:55+00:00
Wirehaired Terrier                  2019-09-19 22:52:45+00:00
Xoloitzcuintli / Mexican Hairless   2019-09-08 11:15:54+00:00
Yellow Labrador Retriever           2019-09-20 06:30:27+00:00
Yorkshire Terrier                   2019-09-20 16:56:38+00:00
Name: posted, Length: 216, dtype: datetime64[ns, UTC]

5. For each state, compute the sex imbalance, that is the difference between male and female dogs. In which state this imbalance is largest?


In [22]:
dogs_by_state = pd.DataFrame()

In [23]:
dogs_by_state['female_dogs'] = dogs[dogs['sex']=='Female'].groupby(['contact_state']).count()['sex']

In [24]:
dogs_by_state['male_dogs'] = dogs[dogs['sex']=='Male'].groupby(['contact_state']).count()['sex']

In [25]:
dogs_by_state['difference'] = (dogs_by_state['male_dogs'] - dogs_by_state['female_dogs']).abs()

In [26]:
dogs_by_state.sort_values(by='difference', ascending=False)

,female_dogs,male_dogs,difference
contact_state,,,
OH,1234,1439,205
IN,851,1029,178
VA,1451,1610,159
MD,679,815,136
TN,826,945,119
NY,1945,2062,117
AZ,1067,1181,114
CA,777,887,110
KY,507,616,109


6. For each pair (age, size), determine the average duration of the stay and the average cost of stay.


In [27]:
average = dogs[['age', 'size', 'stay_duration', 'stay_cost']].groupby(['age', 'size']).mean()
average.rename(columns = {'stay_duration':'Avgerage duration', 'stay_cost':'Avgerage stay cost'})

Avgerage duration  Avgerage stay cost
age    size                                              
Adult  Extra Large          89.015414          232.591561
       Large                89.531943          238.661141
       Medium               89.421036          238.258977
       Small                89.407479          238.974838
Baby   Extra Large          87.032967          237.180879
       Large                89.701564          238.698827
       Medium               89.577668          237.108131
       Small                89.958291          239.083810
Senior Extra Large          88.861111          235.232361
       Large                88.984298          237.507364
       Medium               89.810052          238.514615
       Small                89.073626          238.282286
Young  Extra Large          90.586345          245.835582
       Large                90.104206          238.149506
       Medium               89.515123          239.304603
       Small                89.814275          241.540069

7. Find the dogs involved in at least 3 travels. Also list the breed of those dogs.


In [28]:
df = pd.merge(how='inner', left=dogs, right=travels, on='id')

In [29]:
df = df.groupby(['id', 'breed_primary'])['id'].count().reset_index(name='travels')

In [30]:
more_than_3 = df[df['travels']>2]

In [31]:
more_than_3

,id,breed_primary,travels
5,16657005,Pit Bull Terrier,4
9,20905974,Chow Chow,5
17,24894870,Hound,4
18,24894894,Hound,4
55,33218331,Alaskan Malamute,7
...,...,...,...
4110,46042569,Labrador Retriever,3
4111,46042587,Labrador Retriever,3
4112,46042618,Labrador Retriever,3
4113,46043099,Labrador Retriever,3


8. Fix the travels table so that the correct state is computed from the manual and the found fields. If manual is not missing, then it overrides what is stored in found

In [32]:
travels['found'] = np.where(~travels['manual'].isnull(),travels['manual'],travels['found'])

In [33]:
travels[['found','manual']]

,found,manual
0,Arkansas,NaN
1,Bahamas,Bahamas
2,Maryland,Maryland
3,Adaptil,NaN
4,Afghanistan,NaN
...,...,...
6189,WV,NaN
6190,Wyoming,NaN
6191,Yazmin,NaN
6192,Ohio,Ohio


9. For each state, compute the ratio between the number of travels and the population.


Prepping the dataset

In [34]:
pop = pd.read_csv('NST-EST2021-POP.csv', header=None, names=['state', 'population'])

In [35]:
#serve eliminare il punto che separa le miglialia per far riconoscere l'intero come tale e non come stringa
pop['population'] = pop['population'].apply(lambda x: str(x).replace('.', '')).astype(int)

In [36]:
#we need to prep the dataframe by adding a column for state code (this will facilitate us later).
pop['code'] = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

#fonte: https://gist.github.com/JeffPaine/3083347

In [37]:
#calcolo il numero di viaggi per stato. reset_index evita che la prima riga venga utilizzata come nome della colonna. rinomino le colonne
travels_per_state = travels.groupby('contact_state').count()[['index']].reset_index(level=0).rename(columns={'index':'travels'})

Now can proceed by merging and computing the ratio

In [38]:
#merge tra popolazione e numero di viaggi
pop = pd.merge(how='left', left=pop, right=travels_per_state, left_on='code', right_on='contact_state').drop(columns='contact_state')

In [39]:
pop['travels'] = pop['travels'].fillna(0)

In [40]:
#calcoliamo la percentuale anziché la ratio per evitare che il numero venga espresso in notazione esponenziale
pop['percentage_travels_pop'] = pop['travels'] / pop['population'] * 100

In [41]:
pop['travels'] = pop['travels'].astype(int)

In [42]:
pop[['code', 'population','travels','percentage_travels_pop']].head()

,code,population,travels,percentage_travels_pop
0,AL,5024279,75,0.001493
1,AK,733391,0,0.000000
2,AZ,7151502,70,0.000979
3,AR,3011524,10,0.000332
4,CA,39538223,28,0.000071


10. For each dog, compute the number of days from the posted day to the day of last access.


In [43]:
dogs['accessed'] = pd.to_datetime(dogs['accessed'])

In [44]:
#siccome accessed è solo una data (senza orario), rimuovo l'orario anche da posted
dogs['posted_day'] = pd.to_datetime(dogs['posted'].apply(lambda d: d.date()))

In [45]:
dogs['difference'] = (dogs['accessed'] - dogs['posted_day'])

In [46]:
dogs[['id', 'posted_day', 'accessed', 'difference']]

,id,posted_day,accessed,difference
0,46042150,2019-09-20,2019-09-20,0 days
1,46042002,2019-09-20,2019-09-20,0 days
2,46040898,2019-09-20,2019-09-20,0 days
3,46039877,2019-09-20,2019-09-20,0 days
4,46039306,2019-09-20,2019-09-20,0 days
...,...,...,...,...
58175,44605893,2019-05-03,2019-09-20,140 days
58176,44457061,2019-04-13,2019-09-20,160 days
58177,42865848,2018-09-27,2019-09-20,358 days
58178,42734734,2018-09-12,2019-09-20,373 days


11. Partition the dogs according to the number of weeks from the posted day to the day of last access.

In [47]:
#aggiungere colonna contenente il numero di settimane calcolato come difference/7, e arrotondare all'intero inferiore
dogs['weeks_posted_to_access'] = np.floor((dogs['difference'].dt.days/7)).astype(int)

In [48]:
#lista con i possibili n di settimane
weeks = list(set(dogs['weeks_posted_to_access']))

In [49]:
weeks_dict = {}

In [50]:
#fare un dizionario di dizionario per vedere, in base ad ogni settimana, il totale e l'elenco dei cani 
for difference in weeks:
    #lista dei cani che hanno come difference quel valore specifico
    dog_list = list(set(dogs[dogs['weeks_posted_to_access'] == difference]['id']))
    #creo dizionario per settimane con chiavi il totale dei cani e la lista degli id
    weeks_dict[difference] = {'dog_count': len(dog_list), 'dog_list': dog_list}

12. Find for duplicates in the dogs dataset. Two records are duplicates if they have (1) same breeds and sex, and (2) they share at least 90% of the words in the description field. Extra points if you find and implement a more refined way for determining if two rows are duplicates.

In [51]:
#mettere ogni descrizione in stampatello minuscolo
dogs['description'] = dogs['description'].str.lower()

In [52]:
#inizializzare dizionari per riempirli coi bag of words
dogs['description_bow'] = np.nan
description_bows = [{} for index in dogs.index.tolist()]

In [53]:
description_index = dogs.columns.get_loc('description')
description_bow_index = dogs.columns.get_loc('description_bow')

In [54]:
#carico dizionario description_bows
for index in dogs.index.tolist():
    row_description = dogs.iloc[index, description_index] 
    bow = {}
    for word in row_description.split():
        if word not in bow:
            bow[word] = 1
        else:
            bow[word] += 1         
    description_bows[index] = bow

In [55]:
name_index = dogs.columns.get_loc('name')
breed_primary_index = dogs.columns.get_loc('breed_primary')
sex_index = dogs.columns.get_loc('sex')
age_index = dogs.columns.get_loc('age')

breeds = list(dogs.iloc[:, breed_primary_index])
sexes = list(dogs.iloc[:, sex_index])
names = list(dogs.iloc[:, name_index])
ages = list(dogs.iloc[:, age_index])

In [56]:
def compare_dogs_descriptions(d1, d2):
    total_words = 0
    common_words = 0
    
    for word in set(d1.keys()).union(set(d2.keys())):
        if (word in d1) and (word in d2):
            total_words += max(d1[word], d2[word])
            common_words += min(d1[word], d2[word])
        elif word in d1:
            total_words+=d1[word]
        else:
            total_words+=d2[word]
            
    if total_words == 0:
        return False
    
    return common_words/total_words

In [57]:
def compare_names(name1, name2):
    longest_name, shortest_name = (name1, name2) if len(name1) > len(name2) else (name2, name1)
    return longest_name.lower().find(shortest_name.lower()) != -1

In [58]:
#creare dei dizionari contenenti gruppi di cani che hanno stessa razza e stesso sesso
groups = {}

for index in dogs.index.tolist():
    key = "{}|{}".format(breeds[index], sexes[index])
    if key not in groups:
        groups[key] = [index]
    else:
        groups[key].append(index)

In [59]:
duplicates = []
initial_time = time.time()

#per ogni gruppo di cani con stesso sesso e stessa razza
for group in groups.values():
    #per ogni elemento del gruppo
    for group_first_index, df_first_index in enumerate(group[:len(group)-1]):
        #confrontare ogni elemento coi successivi
        for _, df_second_index in enumerate(group[group_first_index+1:]):
            #potremo dire che due record sono duplicati se
            #l'attributo 'name' dei due record è:
            if compare_names(names[df_first_index], names[df_second_index]):
                #nel caso la descrizione sia vuota, il nome deve essere uguale per entrambi i cani per poter dire che si tratta di un duplicato
                if (len(description_bows[df_first_index].keys()) + len(description_bows[df_second_index].keys())==0) and "".join(names[df_first_index].lower().split())=="".join(names[df_second_index].lower().split()):
                    duplicates.append((df_first_index, df_second_index))
                #nel caso due record condividano il 90% o più della descrizione, è sufficiente che il nome più corto sia contenuto 
                #nel nome più lungo per dichiararli duplicati
                elif compare_dogs_descriptions(description_bows[df_first_index], description_bows[df_second_index]) >= 0.9:
                    duplicates.append((df_first_index, df_second_index))
                
print("Execution time: ", time.time()-initial_time, "s")
print("Gruppi di duplicati: " + str(len(duplicates)))

Execution time:  49.75074601173401 s
Gruppi di duplicati: 1789


In [60]:
#eliminare gli id che compaiono più di una volta nella lista dei duplicati
duplicate_list = list(set(itertools.chain(*duplicates)))

In [61]:
df_duplicates = pd.DataFrame(columns = ['id', 'name', 'description','breed_primary', 'sex'])
df_duplicates['id'] = duplicate_list

In [62]:
count = 0
for i in df_duplicates['id']:
    df_duplicates.loc[count, 'name'] = dogs.loc[i, 'name']
    df_duplicates.loc[count, 'description'] = dogs.loc[i, 'description']
    df_duplicates.loc[count, 'breed_primary'] = dogs.loc[i, 'breed_primary']
    df_duplicates.loc[count, 'sex'] = dogs.loc[i, 'sex']
    count += 1

In [63]:
df_duplicates.sort_values(['breed_primary', 'name'])

,id,name,description,breed_primary,sex
1920,56148,Titan,"my name is titan iâm 2 years old, 96lbs and ...",Akita,Male
1921,56149,Titan,"my name is titan iâm 2 years old, 96lbs and ...",Akita,Male
1757,22962,ANGEL,,American Bulldog,Female
469,26169,Angel,,American Bulldog,Female
25,94,BELLA,,American Bulldog,Female
...,...,...,...,...,...
1343,21214,Ruby Tuesday,you must apply on our website\nhttp://www.york...,Yorkshire Terrier,Female
416,17811,Snagglepuss,you must apply on our website\nhttp://www.york...,Yorkshire Terrier,Female
1369,21364,Snagglepuss,you must apply on our website\nhttp://www.york...,Yorkshire Terrier,Female
368,17610,Tank,you must apply on our website\nhttp://www.york...,Yorkshire Terrier,Male
